In [1]:
import pandas as pd
import numpy as np
import codecs
from rdkit import Chem
from rdkit.Chem.SaltRemover import SaltRemover
from subword_nmt.apply_bpe import BPE

# Preprocess Omics Data 

In [2]:
cell_list = pd.read_csv('/root/lanyun-tmp/Project/SynergyX/cell_info_used.csv')['depmap_id'].values
gene_list = pd.read_csv('./raw_data/gene_4079.csv')['entrez_id'].values
cell_list

array(['A2058', 'NCI-H226', 'K-562', 'CCRF-CEM', 'MDA-MB-468', 'IGROV1',
       'SNB-75', 'SK-MEL-28', 'SW-620', 'MDA-MB-231', 'SF-268', 'HOP-62',
       'NCI-H522', 'NCIH23', 'MOLT-4', 'PC-3', 'OVCAR-5', 'T-47D',
       'UACC-257', 'NCI-H322M', 'SF-539', 'HT29', 'KM12', 'UACC62',
       'BT-549', 'HCC-2998', 'SN12C', 'HS 578T', 'LOX IMVI', 'M14',
       'A549', 'SNB-19', 'A498', 'SK-OV-3', 'CAKI-1', '786-0', 'HCT-15',
       'NCI-H460', 'TK-10', 'MALME-3M', 'COLO 205', 'OVCAR-8', 'UO-31',
       'HOP-92', 'EKVX', 'MCF7', 'ACHN', 'SK-MEL-2', 'SK-MEL-5', 'SF-295',
       'OVCAR-4'], dtype=object)

In [3]:
# 'tanh_norm'
def normalization(data): 
    std1 = np.nanstd(data, axis=0)  
    data = np.ascontiguousarray(data)
    means1 = np.mean(data, axis=0) 

    data = (data-means1)/std1
    data = np.tanh(data)

    data[np.isnan(data)] = 0
    
    return data

In [4]:
def process_gene(df):
    # 填充缺失基因列
    for i in gene_list.astype('str'):
        if i not in df.columns:
            df[i] = 0.

    # 去除重复 index（保留第一次出现的）
    df = df[~df.index.duplicated(keep='first')]

    # 补全缺失的细胞行
    mean_arr = df.mean()
    for j in cell_list:
        if j not in df.index:
            df.loc[j, :] = mean_arr

    # 按基因列顺序和 cell_list 顺序对齐
    df = df[gene_list.astype('str')]
    df = df.loc[cell_list, :]

    # 标准化并输出诊断信息
    df_arr = normalization(df.values)
    print("✅ 去重后 df 行数:", df.shape[0])
    print("前5个细胞系:", df.index[:5].tolist())
    print("是否有重复:", df.index.duplicated().sum())
    print("最终数组形状:", df_arr.shape)

    return df_arr

met_raw = pd.read_csv('./raw_data/omics_data/cell_met_raw.csv', index_col=0)
met_norm = process_gene(met_raw)


/tmp/ipykernel_1053607/3753562466.py:5: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[i] = 0.
/tmp/ipykernel_1053607/3753562466.py:5: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[i] = 0.
/tmp/ipykernel_1053607/3753562466.py:5: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[i] = 0.
/tmp/ipyke

✅ 去重后 df 行数: 51
前5个细胞系: ['A2058', 'NCI-H226', 'K-562', 'CCRF-CEM', 'MDA-MB-468']
是否有重复: 0
最终数组形状: (51, 4079)


/tmp/ipykernel_1053607/2782279984.py:7: RuntimeWarning: divide by zero encountered in divide
  data = (data-means1)/std1
/tmp/ipykernel_1053607/2782279984.py:7: RuntimeWarning: invalid value encountered in divide
  data = (data-means1)/std1


In [5]:
exp_raw = pd.read_csv('./raw_data/omics_data/cell_exp_raw.csv', index_col=0)
mut_raw = pd.read_csv('./raw_data/omics_data/cell_mut_raw.csv', index_col=0)
cn_raw = pd.read_csv('./raw_data/omics_data/cell_cn_raw.csv', index_col=0)
eff_raw = pd.read_csv('./raw_data/omics_data/cell_eff_raw.csv', index_col=0)
dep_raw = pd.read_csv('./raw_data/omics_data/cell_dep_raw.csv', index_col=0)
exp_raw

,7105,64102,8813,57147,55732,2268,3075,2519,2729,4800,...,89839,1124.1,54816.1,11046.1,114483834,548644,114483833,647264,106865373,102724657
DepMap_ID,,,,,,,,,,,,,,,,,,,,,
ACH-000001,5.823495,0.000000,7.369292,2.100978,4.221877,0.042644,0.910733,5.672991,4.676380,4.013462,...,1.545968,0.189034,0.275007,0.000000,0.545968,4.424922,0.000000,0.000000,0.124328,0.000000
ACH-000002,0.189034,0.000000,5.633431,1.263034,3.152183,4.189034,0.163499,4.161888,4.139961,2.799087,...,1.761285,0.000000,0.056584,0.000000,0.799087,3.748461,0.000000,0.000000,0.189034,0.000000
ACH-000003,6.035624,0.084064,7.744767,1.851999,3.895303,0.000000,0.056584,6.597978,4.734981,4.141596,...,1.895303,0.344828,0.042644,0.028569,2.914565,5.350144,0.111031,0.000000,0.070389,0.000000
ACH-000004,2.599318,0.000000,5.310340,2.467279,3.926948,0.918386,4.957915,3.912650,4.878725,4.971314,...,2.704872,0.286881,0.070389,0.454176,2.257011,6.182692,0.000000,0.000000,0.782409,0.000000
ACH-000005,3.051372,0.000000,5.754888,2.969012,5.322289,0.214125,5.762880,4.152995,5.532629,5.330558,...,2.580145,0.454176,0.137504,0.084064,1.560715,5.469886,0.000000,0.000000,0.042644,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ACH-002508,5.556736,0.000000,6.861955,2.220330,2.843984,0.014355,4.788164,5.529509,4.036503,3.463361,...,1.485427,0.000000,0.176323,0.000000,2.056584,3.777157,0.000000,0.000000,0.286881,0.000000
ACH-002509,5.548128,0.344828,6.132166,3.249445,4.156235,0.000000,1.981853,5.844737,3.471187,4.622345,...,0.948601,0.014355,0.275007,0.000000,0.807355,5.042644,0.000000,0.000000,1.422233,0.000000
ACH-002510,4.052242,0.000000,6.295539,1.922198,2.364572,0.000000,1.495695,6.338424,3.768714,4.216455,...,0.176323,0.028569,0.042644,0.028569,2.456806,5.454505,0.000000,0.042644,1.454176,0.042644


In [6]:
exp_norm = process_gene(exp_raw)
mut_norm = process_gene(mut_raw)
cn_norm = process_gene(cn_raw)
eff_norm = process_gene(eff_raw)
dep_norm = process_gene(dep_raw)
exp_norm

/tmp/ipykernel_1053607/2782279984.py:7: RuntimeWarning: divide by zero encountered in divide
  data = (data-means1)/std1
/tmp/ipykernel_1053607/2782279984.py:7: RuntimeWarning: invalid value encountered in divide
  data = (data-means1)/std1
/tmp/ipykernel_1053607/3753562466.py:5: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[i] = 0.
/tmp/ipykernel_1053607/3753562466.py:5: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[i] = 0.
/tmp/ipykernel_1053607/3753562466.py:5: PerformanceWarning: DataFrame is highly fragmented.  This i

✅ 去重后 df 行数: 51
前5个细胞系: ['A2058', 'NCI-H226', 'K-562', 'CCRF-CEM', 'MDA-MB-468']
是否有重复: 0
最终数组形状: (51, 4079)


/tmp/ipykernel_1053607/3753562466.py:5: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[i] = 0.
/tmp/ipykernel_1053607/3753562466.py:5: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[i] = 0.
/tmp/ipykernel_1053607/3753562466.py:5: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[i] = 0.
/tmp/ipyke

✅ 去重后 df 行数: 51
前5个细胞系: ['A2058', 'NCI-H226', 'K-562', 'CCRF-CEM', 'MDA-MB-468']
是否有重复: 0
最终数组形状: (51, 4079)


/tmp/ipykernel_1053607/2782279984.py:7: RuntimeWarning: divide by zero encountered in divide
  data = (data-means1)/std1
/tmp/ipykernel_1053607/2782279984.py:7: RuntimeWarning: invalid value encountered in divide
  data = (data-means1)/std1
/tmp/ipykernel_1053607/3753562466.py:5: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[i] = 0.
/tmp/ipykernel_1053607/3753562466.py:5: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[i] = 0.
/tmp/ipykernel_1053607/3753562466.py:5: PerformanceWarning: DataFrame is highly fragmented.  This i

...

✅ 去重后 df 行数: 51
前5个细胞系: ['A2058', 'NCI-H226', 'K-562', 'CCRF-CEM', 'MDA-MB-468']
是否有重复: 0
最终数组形状: (51, 4079)


/tmp/ipykernel_1053607/3753562466.py:5: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[i] = 0.
/tmp/ipykernel_1053607/3753562466.py:5: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[i] = 0.
/tmp/ipykernel_1053607/3753562466.py:5: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[i] = 0.
/tmp/ipyke

✅ 去重后 df 行数: 51
前5个细胞系: ['A2058', 'NCI-H226', 'K-562', 'CCRF-CEM', 'MDA-MB-468']
是否有重复: 0
最终数组形状: (51, 4079)


/tmp/ipykernel_1053607/2782279984.py:7: RuntimeWarning: divide by zero encountered in divide
  data = (data-means1)/std1
/tmp/ipykernel_1053607/2782279984.py:7: RuntimeWarning: invalid value encountered in divide
  data = (data-means1)/std1


array([[-0.9999092 , -1.        ,  0.99998771, ...,  0.        ,
        -0.99505475,  0.99998771],
       [-0.9999092 , -1.        ,  0.99998771, ...,  0.        ,
        -0.99505475,  0.99998771],
       [-0.9999092 , -1.        ,  0.99998771, ...,  0.        ,
        -0.99505475,  0.99998771],
       ...,
       [-0.9999092 , -1.        ,  0.99998771, ...,  0.        ,
        -0.99505475,  0.99998771],
       [-0.9999092 , -1.        ,  0.99998771, ...,  0.        ,
        -0.99505475,  0.99998771],
       [-0.9999092 , -1.        ,  0.99998771, ...,  0.        ,
        -0.99505475,  0.99998771]])

In [ ]:
met_raw = pd.read_csv('./raw_data/omics_data/cell_met_raw.csv', index_col=0)
met_norm = process_gene(met_raw)


是否含 NaN： False
NaN 数量： 0


In [7]:
print("exp:", exp_norm.shape)
print("met:", met_norm.shape)
print("mut:", mut_norm.shape)
print("cn :", cn_norm.shape)
print("eff:", eff_norm.shape)
print("dep:", dep_norm.shape)



exp: (51, 4079)
met: (51, 4079)
mut: (51, 4079)
cn : (51, 4079)
eff: (51, 4079)
dep: (51, 4079)


In [8]:
# stack six-omics
exp_mut_cn_eff_dep_met = np.dstack((exp_norm,mut_norm,cn_norm,eff_norm,dep_norm,met_norm))
exp_mut_cn_eff_dep_met

array([[[-0.9999092 , -0.99505475,  0.76159416, -1.        ,
          0.99999997,  1.        ],
        [-1.        , -1.        ,  0.99505475,  0.        ,
          0.9981779 ,  1.        ],
        [ 0.99998771, -0.99998771, -1.        ,  1.        ,
         -0.99998771, -0.99998771],
        ...,
        [ 0.        ,  0.        ,  1.        ,  0.        ,
          0.        ,  0.        ],
        [-0.99505475,  0.        ,  1.        ,  0.        ,
          0.        ,  0.        ],
        [ 0.99998771,  0.        , -0.9993293 ,  0.        ,
          0.        ,  0.        ]],

       [[-0.9999092 , -0.99505475,  0.76159416, -1.        ,
          0.99999997,  1.        ],
        [-1.        , -1.        ,  0.99505475,  0.        ,
          0.9981779 ,  1.        ],
        [ 0.99998771, -0.99998771, -1.        ,  1.        ,
         -0.99998771, -0.99998771],
        ...,
        [ 0.        ,  0.        ,  1.        ,  0.        ,
          0.        ,  0.        ],
  

In [9]:
# 
exp_mut_cn_eff_dep_met_dict = dict(zip(cell_list,exp_mut_cn_eff_dep_met))
exp_mut_cn_eff_dep_met_dict

{'A2058': array([[-0.9999092 , -0.99505475,  0.76159416, -1.        ,  0.99999997,
          1.        ],
        [-1.        , -1.        ,  0.99505475,  0.        ,  0.9981779 ,
          1.        ],
        [ 0.99998771, -0.99998771, -1.        ,  1.        , -0.99998771,
         -0.99998771],
        ...,
        [ 0.        ,  0.        ,  1.        ,  0.        ,  0.        ,
          0.        ],
        [-0.99505475,  0.        ,  1.        ,  0.        ,  0.        ,
          0.        ],
        [ 0.99998771,  0.        , -0.9993293 ,  0.        ,  0.        ,
          0.        ]]),
 'NCI-H226': array([[-0.9999092 , -0.99505475,  0.76159416, -1.        ,  0.99999997,
          1.        ],
        [-1.        , -1.        ,  0.99505475,  0.        ,  0.9981779 ,
          1.        ],
        [ 0.99998771, -0.99998771, -1.        ,  1.        , -0.99998771,
         -0.99998771],
        ...,
        [ 0.        ,  0.        ,  1.        ,  0.        ,  0.        ,
    

In [10]:
np.save('/root/lanyun-tmp/Project/SynergyX/data/0_cell_data/4079g/4cellGraphs_exp_mut_cn_eff_dep_met_4079_genes_norm.npy',exp_mut_cn_eff_dep_met_dict)

# Generate Drug Substructure Encoding

In [3]:
def process_smi(smi):

    if ';' in smi:
        smi = smi.split(';')[0].split(';')[0].split(';')[0]
    m = Chem.MolFromSmiles(smi)
    remover = SaltRemover()

    try:
        moll, deleted = remover.StripMolWithDeleted(m)
        Chem.AddHs(moll)
    except Exception as e:
        print('except:',e)

    canonical_smi = Chem.MolToSmiles(moll)
    return canonical_smi

In [4]:
def drug2emb_encoder(smile):
        
        vocab_path = '/root/lanyun-tmp/Project/SynergyX/data/raw_data/drug_substructure/drug_codes.txt'
        sub2idx = np.load('/root/lanyun-tmp/Project/SynergyX/data/raw_data/drug_substructure/sub2idx.npy',allow_pickle=True).item()     
        bpe_codes_drug = codecs.open(vocab_path)
        dbpe = BPE(bpe_codes_drug, merges=-1, separator='')

        max_d = 50
        try:
            t1 = dbpe.process_line(smile).split()  
        except:
            print('this molecular is error:')
            print(smile)

        try:
            i1 = np.asarray([sub2idx[i] for i in t1])  
        except:
            i1 = np.array([0])

        l = len(i1)
        if l < max_d:
            i = np.pad(i1, (0, max_d - l), 'constant', constant_values=0)
            input_mask = ([1] * l) + ([0] * (max_d - l))
        else:
            i = i1[:max_d]
            input_mask = [1] * max_d

        return i, np.asarray(input_mask)

In [5]:
# Obtain SMILES of drugs and convert them to canonical SMILES using the Rdkit package
drug_smiles = pd.read_csv('/root/lanyun-tmp/Project/SynergyX/data/raw_data/drug_info_used.csv',index_col=0)['canonical_smiles'].values
drug_smiles = [process_smi(smi) for smi in drug_smiles] 
drug_smiles = np.unique(drug_smiles)
drug_smiles

array(['C#CC1(O)CCC2C3CCC4=CC(=O)CCC4C3CCC21CC',
       'C#CC1(O)CCC2C3CCc4cc(O)ccc4C3CCC21C',
       'C#CCC(Cc1cnc2nc(N)nc(N)c2n1)c1ccc(C(=O)NC(CCC(=O)O)C(=O)O)cc1',
       'C#CCN(C)Cc1ccccc1', 'C#Cc1cccc(Nc2ncnc3cc(OCCOC)c(OCCOC)cc23)c1',
       'C1CN1c1nc(N2CC2)nc(N2CC2)n1', 'C1CNCCN1',
       'C=C1CC2C(CCC3(C)C(=O)CCC23)C2(C)C=CC(=O)C=C12',
       'C=CC1CN2CCC1CC2C(O)c1ccnc2ccc(OC)cc12',
       'C=CCC1C=C(C)CC(C)CC(OC)C2OC(O)(C(=O)C(=O)N3CCCCC3C(=O)OC(C(C)=CC3CCC(O)C(OC)C3)C(C)C(O)CC1=O)C(C)CC2OC',
       'C=CCn1c(=O)c2cnc(Nc3ccc(N4CCN(C)CC4)cc3)nc2n1-c1cccc(C(C)(C)O)n1',
       'CC#CCC(C)C(O)C=CC1C(O)CC2CC(=CCCCC(=O)O)CC21',
       'CC(=Cc1csc(C)n1)C1CC2OC2(C)CCCC(C)C(O)C(C)C(=O)C(C)(C)C(O)CC(=O)N1',
       'CC(=O)NC1C(O)CC(O)(C(=O)O)OC1C(O)C(O)CO', 'CC(=O)NCCCS(=O)(=O)O',
       'CC(=O)NO', 'CC(=O)Nc1nnc(S(N)(=O)=O)s1',
       'CC(=O)OC1(C(C)=O)CCC2C3C=C(C)C4=CC(=O)CCC4(C)C3CCC21C',
       'CC(=O)OC1(C(C)=O)CCC2C3CCC4=CC(=O)CCC4=C3C(c3ccc(N(C)C)cc3)CC21C',
       'CC(=O)OC12COC1C

In [6]:
drugSmile_drugSubEmbed = { k:drug2emb_encoder(k) for k in drug_smiles}
drugSmile_drugSubEmbed

{'C#CC1(O)CCC2C3CCC4=CC(=O)CCC4C3CCC21CC': (array([ 43,  17, 192,  96, 103, 122, 485,   8, 412, 103, 232, 485,  38,
         135,  43,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
           0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
           0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0]),
  array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0])),
 'C#CC1(O)CCC2C3CCc4cc(O)ccc4C3CCC21C': (array([  43,   17,  192,   96,  103,  122,  169, 2173, 2120,  485,   38,
           29,   43,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0]),
  array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0,

In [7]:
import os
import numpy as np

save_path = '/root/lanyun-tmp/Project/SynergyX/data/1_drug_data/4drugSmile_drugSubEmbed.npy'

# 如果目录不存在，先创建目录
os.makedirs(os.path.dirname(save_path), exist_ok=True)

# 如果文件不存在则保存
if not os.path.exists(save_path):
    np.save(save_path, drugSmile_drugSubEmbed)
    print(f"✅ 已保存 drugSmile_drugSubEmbed 到 {save_path}")
else:
    print(f"ℹ️ 文件已存在：{save_path}，跳过保存")


✅ 已保存 drugSmile_drugSubEmbed 到 /root/lanyun-tmp/Project/SynergyX/data/1_drug_data/4drugSmile_drugSubEmbed.npy


In [8]:
from rdkit import Chem
from rdkit.Chem.SaltRemover import SaltRemover
import pandas as pd
import numpy as np
import os
import codecs
from tqdm import tqdm

In [9]:
# 环境准备
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [10]:
# 路径定义
drug_info_path = '/root/lanyun-tmp/Project/SynergyX/data/raw_data/drug_info_used.csv'
sub2idx_path = '/root/lanyun-tmp/Project/SynergyX/data/raw_data/drug_substructure/sub2idx.npy'
bpe_vocab_path = '/root/lanyun-tmp/Project/SynergyX/data/raw_data/drug_substructure/drug_codes.txt'
save_path = '/root/lanyun-tmp/Project/SynergyX/data/1_drug_data/4drugSmile_drugSubEmbed_canonical.npy'

In [11]:
# 工具函数
remover = SaltRemover()

def process_smi(smi):
    try:
        smi = smi.strip().replace(" ", "").replace("\n", "").replace("\r", "").replace("\t", "")
        if ';' in smi:
            smi = smi.split(';')[0]
        m = Chem.MolFromSmiles(smi)
        if m is None:
            raise ValueError(f"Invalid SMILES: {smi}")
        moll, _ = remover.StripMolWithDeleted(m)
        canonical_smi = Chem.MolToSmiles(moll)
        return canonical_smi
    except Exception as e:
        print(f"[SMILES ERROR] {smi} → {e}")
        return None


In [12]:
# BPE 编码器定义
class BPE:
    def __init__(self, codes, merges=-1, separator='@@'):
        self.bpe_codes = [tuple(item.strip().split()) for (n, item) in enumerate(codes)]
        self.separator = separator
        self.bpe_codes = dict([(pair, i) for (i, pair) in reversed(list(enumerate(self.bpe_codes)))])
        self.cache = {}

    def process_line(self, line):
        word = tuple(line.strip())
        pairs = [(word[i], word[i + 1]) for i in range(len(word) - 1)]
        if not pairs:
            return line
        while True:
            bigram = min(pairs, key=lambda pair: self.bpe_codes.get(pair, float('inf')))
            if bigram not in self.bpe_codes:
                break
            first, second = bigram
            new_word = []
            i = 0
            while i < len(word):
                try:
                    j = word.index(first, i)
                    new_word.extend(word[i:j])
                    i = j
                except:
                    new_word.extend(word[i:])
                    break
                if word[i] == first and i < len(word) - 1 and word[i + 1] == second:
                    new_word.append(first + second)
                    i += 2
                else:
                    new_word.append(word[i])
                    i += 1
            word = tuple(new_word)
            if len(word) == 1:
                break
            else:
                pairs = [(word[i], word[i + 1]) for i in range(len(word) - 1)]
        word = self.separator.join(word)
        return word


In [13]:
# 药物编码函数
def drug2emb_encoder(smile, sub2idx, dbpe, max_d=50):
    try:
        t1 = dbpe.process_line(smile).split()
    except:
        print('Error in BPE processing:', smile)
        t1 = []

    try:
        i1 = np.asarray([sub2idx.get(i, 0) for i in t1])
    except:
        i1 = np.array([0])

    l = len(i1)
    if l < max_d:
        i = np.pad(i1, (0, max_d - l), 'constant', constant_values=0)
        input_mask = ([1] * l) + ([0] * (max_d - l))
    else:
        i = i1[:max_d]
        input_mask = [1] * max_d

    return i, np.asarray(input_mask)

In [14]:
# 加载数据
df = pd.read_csv(drug_info_path, index_col=0)
raw_smiles = df['canonical_smiles'].values
processed_smiles = list(filter(None, [process_smi(s) for s in raw_smiles]))
unique_smiles = np.unique(processed_smiles)

# 初始化编码器
sub2idx = np.load(sub2idx_path, allow_pickle=True).item()
with codecs.open(bpe_vocab_path, encoding='utf-8') as f:
    dbpe = BPE(f, merges=-1, separator='')

In [15]:
# 编码并保存
drugSmile_drugSubEmbed = {
    smi: drug2emb_encoder(smi, sub2idx, dbpe)
    for smi in tqdm(unique_smiles)
}

os.makedirs(os.path.dirname(save_path), exist_ok=True)
np.save(save_path, drugSmile_drugSubEmbed)

100%|██████████| 367/367 [00:00<00:00, 1139.40it/s]
